# CoSim - RTED

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.4.post102.dev0+geca4145'

In [3]:
andes.config_logger(stream_level=20)

In [4]:
sa = andes.load(andes.get_case('kundur/kundur_coi.xlsx'),
                setup=True,
                no_output=True,
                )


Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/kundur/kundur_coi.xlsx"...
Input file parsed in 0.1215 seconds.
System internal structure set up in 0.0219 seconds.


In [5]:
ams.config_logger(stream_level=10)

In [6]:
sp = ams.load(ams.get_case('ieee14/ieee14_rted.xlsx'),
              setup=True,
              )

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee14/ieee14_rted.xlsx"...
Input file parsed in 0.0462 seconds.
Adjust bus index to start from 0.
System set up in 0.0043 seconds.


In [7]:
sp.RTED.run()

Setup model of RTED
RTED pg0 are all zeros, correct to p0.
RTED data check passed.
- Generating symbols for RTED
Set constrs pb: self.constrs["pb"]=cp.sum(self.routine.pd.v) - cp.sum(self.pg) == 0
Set constrs lub: self.constrs["lub"]=self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) - self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
Set constrs llb: self.constrs["llb"]=- self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) + self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
Set constrs rbu: self.constrs["rbu"]=self.routine.prs.v @ self.pru - self.routine.du.v == 0
Set constrs rbd: self.constrs["rbd"]=self.routine.prs.v @ self.prd - self.routine.dd.v == 0
Set constrs rru: self.constrs["rru"]=self.pg + self.pru - self.routine.pmax.v <= 0
Set constrs rrd: self.constrs["rrd"]=-self.pg + self.prd - self.routine.pmin.v <= 0
Set constrs rgu: self.constrs["rgu"]=self.pg - self.routine.pg0.v - self.routine.R10.v <= 0
Set constrs rgd: self.constrs[

In [8]:
sp.RTED.smooth()

Setup model for ACOPF
ACOPF data check passed.
- Generating symbols for ACOPF
Set constrs pb: self.constrs["pb"]=cp.sum(self.routine.pd.v) - cp.sum(self.pg) == 0
ACOPF model set up in 0.0029 seconds.
Adjust bus index to start from 0.
PYPOWER Version 5.1.4, 27-June-2018
 -- AC Optimal Power Flow



Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011


ACOPF completed in 0.3364 seconds with exit code 0.
RTED is smoothed with ACOPF.


Converged!


True

In [ ]:
sa = sp.to_andes(setup=False,
                 addfile=andes.get_case('ieee14/ieee14_wt3.xlsx'),
                 overwrite=True,
                 keep=False,
                 no_output=True,
                 )

sa.add("Toggle", dict(model='SynGen', dev="GENROU_4", t=11.0))

sa.setup()

In [ ]:
sp.dyn.send()

In [ ]:
sa.PFlow.run()

In [ ]:
sp.RTED.pg0

In [ ]:
idx_tg = sa.TGOV1.idx.v
vmax0 = sa.TGOV1.get(src='VMAX', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMAX', attr='v', idx=idx_tg, value=10 * vmax0)
vmin0 = sa.TGOV1.get(src='VMIN', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMIN', attr='v', idx=idx_tg, value=np.zeros_like(vmin0))

In [ ]:
sa.TDS.config.tf = 10
sa.TDS.run()

In [ ]:
# sa.TDS.config.tf = 20
# sa.TDS.run()

In [ ]:
sp.dyn.sa

In [ ]:
sp.dyn.send()

In [ ]:
sp.dyn.receive()